In [1]:
!pip install -U langchain-community langchain faiss-cpu sentence-transformers transformers pandas


In [2]:
import pandas as pd

df = pd.read_csv("Training Dataset.csv")

# Convert rows to readable strings
texts = []
for _, row in df.iterrows():
    row_text = ", ".join([f"{col}: {row[col]}" for col in df.columns])
    texts.append(row_text)

print("Sample row text:\n", texts[0])

# Load poem text
with open("IF.txt", "r", encoding="utf-8") as f:
    poem_text = f.read()

# Optionally chunk long poem
from langchain.text_splitter import CharacterTextSplitter

poem_chunks = CharacterTextSplitter(chunk_size=500, chunk_overlap=0).split_text(poem_text)

# Add to main text list
texts.extend(poem_chunks)


Sample row text:
 Loan_ID: LP001002, Gender: Male, Married: No, Dependents: 0, Education: Graduate, Self_Employed: No, ApplicantIncome: 5849, CoapplicantIncome: 0.0, LoanAmount: nan, Loan_Amount_Term: 360.0, Credit_History: 1.0, Property_Area: Urban, Loan_Status: Y


In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Chunk data
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = splitter.create_documents(texts)

# Embedding model (lightweight & fast)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create vector store
vectorstore = FAISS.from_documents(documents, embedding)


/tmp/ipython-input-3-4253428102.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models o

In [4]:
from transformers import pipeline

# FLAN-T5 is a free and friendly model for Q&A
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256)


Device set to use cuda:0


In [5]:
def answer_question(query):
    # Step 1: Retrieve relevant chunks
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in docs])

    # Step 2: Generate answer using LLM
    prompt = f"Answer the question based on the following data:\n{context}\n\nQuestion: {query}"
    result = llm_pipeline(prompt)[0]['generated_text']
    return result


#Testing chatbot


In [7]:
while True:
    query = input("Ask a question (or 'exit'): ")
    if query.lower() == "exit":
        break
    response = answer_question(query)
    print("Answer:", response)


Ask a question (or 'exit'): Who is the author of the poem IF?
Answer: Rudyard Kipling
Ask a question (or 'exit'): How many stanzas are there in the poem?
Answer: Four stanzas of eight lines each, written in iambic pentameter, creating a rhythmic, conversational flow. The repeated “If” builds a conditional structure leading to the final resolution. Stanza 4: If you can talk with crowds and keep your virtue, Or walk with Kings—nor lose the common touch, If neither foes nor loving friends can hurt you, If all men count with you, but none too much; If you can fill the unforgiving minute With sixty seconds’ worth of distance run, Yours is the Earth and everything that’s in it, And—which is more—you’ll be a Man, my son!
Ask a question (or 'exit'): What is the meaning of the poem IF?
Answer: profound guidance on navigating life’s challenges with resilience, humility, and integrity
Ask a question (or 'exit'): What is the message of the poem IF?
Answer: the promise that embodying these qualitie

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Answer: N
Ask a question (or 'exit'): What is the gender of LP001003?
Answer: Male, Married: No, Dependents: 0, Education: Graduate, Self_Employed: No, ApplicantIncome: 2378, CoapplicantIncome: 0.0, LoanAmount: 258.0, LoanAmount_Term: 360.0, CreditHistory: 0.0, LoanAmount: 258.0, LoanAmount_Term: 360.0, CreditHistory: 0.0, LoanAmount: 258.0, LoanAmount_Term: 360.0, LoanAmount: 3+, Education: Graduate, Graduate, Self_Employed: No, CoapplicantIncome: 8799, CoapplicantIncome: 0.0, LoanAmount: 258.0, LoanAmount_Term: 360.0, CreditHistory: 0.0, LoanAmount: 258.0, LoanAmount_Term: 360.0, LoanAmount_Term: 360.0, LoanAmount: 3+, Education: Graduate, Graduate, Self_Employed: No, ApplicantIncome: 12500, CoapplicantIncome: 3000.0, LoanAmount: 320.0, Loan_Amount
Ask a question (or 'exit'): How many dependents does LP001014 have?
Answer: 3
Ask a question (or 'exit'): exit
